In [14]:
# Interval = min K

import yfinance as yf
from datetime import datetime
import pandas as pd

# ticker = '^gspc'
ticker = 'tsla'

start_date = datetime(2022, 10, 20, 21)
end_date = datetime(2022, 10, 21, 16)
data = yf.download(ticker, interval = '2m', start = start_date ,end = end_date, auto_adjust = False)
data.index.name = 'Date Time'
data.index = pd.to_datetime(data.index, format = '%m/%d/%Y').strftime('%Y-%m-%d %H:%M:%S')
data.to_csv('data.csv')
print(data)

[*********************100%***********************]  1 of 1 completed
                           Open        High         Low       Close  \
Date Time                                                             
2022-10-20 09:30:00  208.279999  208.500000  205.550003  207.375000   
2022-10-20 09:32:00  207.380005  209.669998  205.800003  208.970001   
2022-10-20 09:34:00  208.960007  209.000000  205.750000  206.570007   
2022-10-20 09:36:00  206.615097  206.880005  204.509995  204.619995   
2022-10-20 09:38:00  204.619995  205.869995  203.270493  203.270493   
...                         ...         ...         ...         ...   
2022-10-20 15:52:00  208.009995  208.479996  207.792603  208.440002   
2022-10-20 15:54:00  208.479996  208.630005  207.669998  207.869995   
2022-10-20 15:56:00  207.850006  208.000000  207.630005  207.649994   
2022-10-20 15:58:00  207.639999  207.720001  207.169998  207.205002   
2022-10-28 16:00:00  228.520004  228.520004  228.520004  228.520004   

       

In [15]:
# Interval = 2 min K

from pyalgotrade import strategy
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.technical import ma
from pyalgotrade.technical import rsi
from pyalgotrade.technical import stoch
from pyalgotrade.bar import Frequency


number_of_shares = 10
kdj = []
kdjd = []
rsi1 = []

def safe_round(value, digits):
    if value is not None:
        value = round(value, digits)
    return value

class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, smaPeriod):
        super(MyStrategy, self).__init__(feed, 100000) # initial equity
        
        # We'll use adjusted close values instead of regular close values.
        # self.setUseAdjustedValues(True)
        self.__sma = ma.SMA(feed[instrument].getCloseDataSeries(), smaPeriod)
        self.__rsi = rsi.RSI(feed[instrument].getCloseDataSeries(), 6)
        self.__kdj = stoch.StochasticOscillator(feed[instrument], period=9, dSMAPeriod=3)
        self.D = self.__kdj.getD()
        self.__position = None
        self.__instrument = instrument

    def onEnterOk(self, position):
        global execInfo
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at $%.2f %s %s %s" % (execInfo.getPrice(),safe_round(self.__kdj[-2], 2),safe_round(self.D[-2], 2),self.__position.getShares()))
        global buy, total_buy
        buy = execInfo.getPrice() * number_of_shares
        total_buy += buy

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at $%.2f %s %s %s" % (execInfo.getPrice(),safe_round(self.__kdj[-2], 2),safe_round(self.D[-2], 2),self.__position.getShares()))
        # self.__position = None
        sell = execInfo.getPrice() * number_of_shares
        global return_per_share, total_sell
        total_sell += sell
        return_per_share = total_sell - total_buy

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__sma[-1] is None:
            return
        
        kdj.append(self.__kdj[-1])
        kdjd.append(self.D[-1])
        rsi1.append(self.__rsi[-1])
        # for i in enumerate(self.__rsi):
        #     print(i)
        # ALL 30 70 (20)(14)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLongStop(self.__instrument, stopPrice=execInfo.getPrice() * 0.995, quantity=number_of_shares, goodTillCanceled=True)
        #     if safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterShortStop(self.__instrument, stopPrice=execInfo.getPrice() * 1.005, quantity=number_of_shares, goodTillCanceled=True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShortStop(self.__instrument, stopPrice=execInfo.getPrice() * 1.005, quantity=number_of_shares, goodTillCanceled=True)
        #     if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLongStop(self.__instrument, stopPrice=execInfo.getPrice() * 0.995, quantity=number_of_shares, goodTillCanceled=True)


        # ALL 30 70 (13)(15)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # ALL 20 80 (8.6)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # ALL 20 80 
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # Just < 30 and > 70 (17)(15)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-1], 2) < 30 and safe_round(self.__rsi[-1], 2) < 30:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1], 2) > 70 and safe_round(self.__rsi[-1], 2) > 70:
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-1], 2) > 70 and safe_round(self.__rsi[-1], 2) > 70 and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1], 2) < 30 and safe_round(self.__rsi[-1], 2) < 30 and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # Just < 20 and > 80 (10)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-1], 2) < 20 and safe_round(self.__rsi[-1], 2) < 20:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1], 2) > 80 and safe_round(self.__rsi[-1], 2) > 80:
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-1], 2) > 80 and safe_round(self.__rsi[-1], 2) > 80 and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1], 2) < 20 and safe_round(self.__rsi[-1], 2) < 20 and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # KDJ cross (15.4)
        # if self.__position is None:
        #     # print(self.__kdj[-1],self.D[-1])
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2):
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        if self.__position is None:
            if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2):
                self.__position = self.enterLong(self.__instrument, number_of_shares, True)
            if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2):
                self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        else:
            if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>" and not self.__position.exitActive():
                self.__position.exitMarket()
                self.__position = self.enterShort(self.__instrument, number_of_shares, True)
            if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>" and not self.__position.exitActive():
                self.__position.exitMarket()
                self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # trial 2 (9)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and safe_round(self.__kdj[-2], 2) < 50 and safe_round(self.__rsi[-2], 2) < 50:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and safe_round(self.__kdj[-2], 2) > 50 and safe_round(self.__rsi[-2], 2) > 50:
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # trial 3 (17)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and safe_round(self.__kdj[-2], 2) < 50 and safe_round(self.__rsi[-2], 2) < 50:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and safe_round(self.__kdj[-2], 2) > 50 and safe_round(self.__rsi[-2], 2) > 50:
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)


def run_strategy(smaPeriod):
    global total_sell, total_buy
    total_buy = 0
    total_sell = 0
    # Load the bar feed from the CSV file
    # feed = quandlfeed.Feed()
    feed = csvfeed.GenericBarFeed(Frequency.MINUTE)
    feed.addBarsFromCSV(ticker,'data.csv')

    # Evaluate the strategy with the feed.
    myStrategy = MyStrategy(feed, ticker, smaPeriod)
    myStrategy.run()
    print("Final portfolio value: $%.2f" % myStrategy.getBroker().getEquity())
    # print('Return = ', return_per_share)

run_strategy(15)


2022-10-20 10:00:00 strategy [INFO] BUY at $204.16 45.07 45.5 -10
2022-10-20 10:02:00 strategy [INFO] SELL at $205.05 58.19 51.62 0
2022-10-20 10:02:00 strategy [INFO] BUY at $205.05 58.19 51.62 10
2022-10-20 10:12:00 strategy [INFO] SELL at $209.45 91.06 91.47 0
2022-10-20 10:12:00 strategy [INFO] BUY at $209.45 91.06 91.47 -10
2022-10-20 10:16:00 strategy [INFO] SELL at $210.88 98.48 91.16 0
2022-10-20 10:16:00 strategy [INFO] BUY at $210.88 98.48 91.16 10
2022-10-20 10:20:00 strategy [INFO] SELL at $211.03 88.98 95.49 0
2022-10-20 10:20:00 strategy [INFO] BUY at $211.03 88.98 95.49 -10
2022-10-20 10:36:00 strategy [INFO] SELL at $208.53 23.69 13.34 0
2022-10-20 10:36:00 strategy [INFO] BUY at $208.53 23.69 13.34 10
2022-10-20 10:50:00 strategy [INFO] SELL at $212.27 93.46 94.56 0
2022-10-20 10:50:00 strategy [INFO] BUY at $212.27 93.46 94.56 -10
2022-10-20 10:58:00 strategy [INFO] SELL at $213.28 97.22 88.58 0
2022-10-20 10:58:00 strategy [INFO] BUY at $213.28 97.22 88.58 10
2022-10

Final portfolio value: $99839.80


In [16]:
import plotly.graph_objs as go


trace1 = go.Scatter(
    x=data.index,
    y=kdj,
    mode='lines',
    name='KDJ K Line'
)

trace2 = go.Scatter(
    x=data.index,
    y=kdjd,
    mode='lines',
    name='KDJ D Line'
)

trace3 = go.Scatter(
    x=data.index,
    y=rsi1,
    mode='lines',
    name='RSI1 Line'
)

layout = go.Layout(
    title='indicators',
    xaxis={'title': "Time"},
    yaxis={'title': "kdj/rsi"}
)

fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()